# clinical ner transformer

In this notebook I use pretrained clinic_bioBert encoder transformer to extract diagnosis and medications (NER and classification) from medical notes data.

### Main Objectives

1. Generate or Load Data

- Option A: Synthetic EMRs (fast to iterate; no licensing).
- Option B: Open datasets (stronger realism): BC5CDR (Diseases/Chemicals): https://huggingface.co/datasets/biocreative_cdr

2. BIO Tag the Data

- Label scheme: B-ENTITY, I-ENTITY, O.

3. Tokenize

- Use the same tokenizer as the model (handles subwords + offset mapping).

4. Load Pretrained Model

- Bio_ClinicalBERT: emilyalsentzer/Bio_ClinicalBERT

5. Fine-Tune

- Objective: token classification (NER).
- Loss: cross-entropy over token labels (ignore specials with -100).

6. Classify Diagnoses & Meds

- Postprocess logits → entity spans.
- Aggregate subwords back to words; merge B-/I- runs.

7. Evaluate

- Metrics: precision / recall / F1 (seqeval).
- Inspect errors (boundary splits, abbreviations, synonyms).

### install and import necessary modules

In [ ]:
! pip install datasets transformers seqeval evaluate